1 - IMPORTANDO BIBLIOTECAS

In [1]:
import pandas as pd
import requests
import sqlite3
from plyer import notification
from datetime import date

2 - ALERTA

In [6]:
def alerta(nivel,base, etapa):
    if nivel == 200:
           nivel = 'API OK'
    else:
           nivel = 'Erro ao acessar a API'

    notification.notify(
        title=nivel,
        message=f'Falha no Carregamento da {base} no endPoint {etapa} {date.today()}',
        app_name='ALERTA',
        timeout=10
        
    )

alerta(2,'API','GET')

3 - PRIMEIRA EXTRAÇÃO - PAÍSES

In [4]:
## EXTRAÇÃO DE TODOS OS PAISES 

url = "https://restcountries.com/v3.1/all"
response_01 = requests.get(url)

if response_01.status_code == 200:
    data_jason_01 = response_01.json()
else:
    alerta(2,'API','GET')
    #print("Erroa ao acessar a API")

paises = pd.DataFrame(data_jason_01)
#paises.head(5)

3.1 - TRATAMENTO DA PRIMEIRA EXTRAÇÃO

In [7]:
## TRATAMENTO DA EXTRAÇÃO 01 TRZENDO SÓ O NECESSÁRIO E DEIXANDO O "NOME" COMO CHAVE PARA O JOIN

nome       = [pais['name']['common'] for pais in data_jason_01]
region     = [pais['region'] for pais in data_jason_01]
population = [pais['population'] for pais in data_jason_01]
independent = [pais.get('independent', None) for pais in data_jason_01]

paises_tratados = pd.DataFrame({
    "Nome": nome,
    "Regiao": region,
    "Populacao": population,
    "Independente": [str(value).replace('True', 'Verdadeiro').replace('False', 'Falso') for value in independent]
    })
#paises_tratados.to_excel('XOFXAF3.xlsx')
paises_tratados.head(2)







,Nome,Regiao,Populacao,Independente
0,Uzbekistan,Asia,34232050,Verdadeiro
1,Austria,Europe,8917205,Verdadeiro


4 - SEGUNDA EXTRAÇÃO - REGIÕES

In [8]:
## EXTRAÇÃO DOS PAISES DO CONTINENTE AFRICANO

url = "https://restcountries.com/v3.1/region/africa"
response_02 = requests.get(url)

if response_02.status_code == 200:
    data_jason_02 = response_02.json()
else:
    alerta(2,'API','GET')
    #print("Erroa ao acessar a API")

regioes = pd.DataFrame(data_jason_02)
#regioes.head(1)

4.1 - TRATAMENTO DA SEGUNDA EXTRAÇÃO

In [9]:
## TRATAMENTO DAS COLUNAS NECESSÁRIAS

nome       = [regiao['name']['common'] for regiao in data_jason_02]
continents = [regiao['continents']for regiao in data_jason_02]
startOfWeek = [regiao['startOfWeek']for regiao in data_jason_02]

regioes_tratados = pd.DataFrame({
    "Nome": nome,
    "Continente": continents,
   # "Inicio de Semana": startOfWeek    
    "Inicio da Semana": [str(value).replace('monday', 'Segunda Feira').replace('tuesday', 'Terça Feira').replace('wednesday', 'Quarta Feira').replace('thursday', 'Quinta Feira').replace('friday', 'Sexta Feira').replace('saturday', 'Sabado').replace('sunday', 'Domingo') for value in startOfWeek]
    })
#regioes.to_excel('XOFXAF3.xlsx')
regioes_tratados.head(2)


 


,Nome,Continente,Inicio da Semana
0,Sierra Leone,[Africa],Segunda Feira
1,Algeria,[Africa],Domingo


5 - TERCEIRA EXTRAÇÃO - MOEDA

In [10]:
## EXTRAÇÃO DOS DADOS DAS MOEDAS

url = "https://restcountries.com/v3.1/currency/cfa"
response_03 = requests.get(url)

if response_03.status_code == 200:
    data_jason_03 = response_03.json()
else:
    alerta(2,'API','GET')
    #print("Erroa ao acessar a API")

moedas = pd.DataFrame(data_jason_03)
#moedas.head(5)



5.1 - TRATAMENTO DA EXTRAÇÃO 3

In [12]:
## TRATAMENTO DAS COLUNAS NECESSÁRIAS SEMPRE DEIXANDO A COLUNA "NOME" PARA USAR NO JOIN 

'''currencies = [moeda['currencies']['XOF']['name'] if 'currencies' in moeda and 'XOF' in moeda['currencies'] else
             moeda['currencies']['XAF']['name'] if 'currencies' in moeda and 'XAF' in moeda['currencies'] else 'None'
             for moeda in data_jason_03]'''
nome       = [moeda['name']['common'] for moeda in data_jason_03]
currencies = [moeda['currencies'] for moeda in data_jason_03]
#currencies = [moeda['currencies']#for moeda in data_jason_03],

moedas_tratados = pd.DataFrame({
    "Nome": nome,
    "Moeda": currencies

    })
#moedas.to_excel('XOFXAF3.xlsx')
moedas_tratados.head(2)

,Nome,Moeda
0,Guinea-Bissau,"{'XOF': {'name': 'West African CFA franc', 'sy..."
1,Ivory Coast,"{'XOF': {'name': 'West African CFA franc', 'sy..."


6 - MANIPULAÇÃO DOS DADOS

In [15]:
## OUTER JOIN DE DOIS DF CRIANDO OUTRO DF
geral2 = paises_tratados.merge(moedas_tratados,on='Nome',how='outer')
#geral2

## OUTER JOIN DO RESULTADO DO RESULTADO DO DF COM O DF DA ULTIMA EXTRAÇÃO
geral3 = geral2.merge(regioes_tratados,on='Nome',how='outer')


## GERANDO XLSX PARA CONFERIR 

geral3.to_excel('geral3.xlsx')
print("Antes: ",geral3.shape)

## DROPANDO AS LINHAS VAZIAS PARA DEIXAR SÓ O RESULTADO 
resultado = pd.read_excel("geral3.xlsx")
print("Antes: ",resultado.shape)
resultado = resultado.dropna()
print("Depois: ",resultado.shape)
resultado






Antes:  (250, 7)
Antes:  (250, 8)
Depois:  (14, 8)


,Unnamed: 0,Nome,Regiao,Populacao,Independente,Moeda,Continente,Inicio da Semana
9,9,Cameroon,Africa,26545864,Verdadeiro,"{'XAF': {'name': 'Central African CFA franc', ...",['Africa'],Segunda Feira
18,18,Central African Republic,Africa,4829764,Verdadeiro,"{'XAF': {'name': 'Central African CFA franc', ...",['Africa'],Segunda Feira
20,20,Equatorial Guinea,Africa,1402985,Verdadeiro,"{'XAF': {'name': 'Central African CFA franc', ...",['Africa'],Segunda Feira
36,36,Mali,Africa,20250834,Verdadeiro,"{'XOF': {'name': 'West African CFA franc', 'sy...",['Africa'],Segunda Feira
42,42,Benin,Africa,12123198,Verdadeiro,"{'XOF': {'name': 'West African CFA franc', 'sy...",['Africa'],Segunda Feira
48,48,Republic of the Congo,Africa,5657000,Verdadeiro,"{'XAF': {'name': 'Central African CFA franc', ...",['Africa'],Segunda Feira
62,62,Chad,Africa,16425859,Verdadeiro,"{'XAF': {'name': 'Central African CFA franc', ...",['Africa'],Segunda Feira
75,75,Togo,Africa,8278737,Verdadeiro,"{'XOF': {'name': 'West African CFA franc', 'sy...",['Africa'],Segunda Feira
81,81,Gabon,Africa,2225728,Verdadeiro,"{'XAF': {'name': 'Central African CFA franc', ...",['Africa'],Segunda Feira
125,125,Niger,Africa,24206636,Verdadeiro,"{'XOF': {'name': 'West African CFA franc', 'sy...",['Africa'],Segunda Feira


BANCO DE DADOS

In [15]:
conn = sqlite3.connect('projeto_final.db')
resultado.to_sql('mundo',conn, if_exists='replace',index=False)
conn.close()